In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

Create a folder under cs

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(64),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(*stats,inplace=True)
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomCrop(64),
    transforms.ToTensor(),
#     transforms.Normalize(*stats,inplace=True)
])

In [4]:
!rmdir /mnt/storage/home/y263song/5_shot/test/.ipynb_checkpoints
!rmdir /mnt/storage/home/y263song/5_shot/train/.ipynb_checkpoints

rmdir: failed to remove '/mnt/storage/home/y263song/5_shot/test/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/mnt/storage/home/y263song/5_shot/train/.ipynb_checkpoints': No such file or directory


In [5]:
train = ImageFolder("/mnt/storage/home/y263song/5_shot/train", 
                    transform = train_transform)
test = ImageFolder("/mnt/storage/home/y263song/5_shot/test",
                   transform = test_transform)

In [6]:
random_seed = 480
torch.manual_seed(random_seed);
val_size = int(len(train) * 0.2)
train_size = len(train) - val_size

train_ds, val_ds = random_split(train, [train_size, val_size])
len(train_ds), len(val_ds)

(88, 22)

In [22]:
batch_size = 20

# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

In [7]:
import torchvision.models as models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.vgg16(pretrained=True)

In [13]:
from torch.optim import lr_scheduler

num_ftrs = model_ft.classifier[0].in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier[0] = nn.Linear(num_ftrs, 2)
#model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [23]:
import time
import copy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in train_dl:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes
            epoch_acc = running_corrects.double() / dataset_sizes
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
from torch.utils.data import Dataset
from skimage.io import imread

class TestDataset(Dataset):
    def __init__(self, images_root):
        self.images_root = images_root
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return 517

    def __getitem__(self, index):
        img_path = f"{self.images_root}/{index}.jpg"
        image = imread(img_path)
        img_tensor = self.transform(image)
        name = torch.tensor(int(index))
        return (img_tensor, name)
test_dataset = TestDataset("/mnt/storage/home/y263song/5_shot/test/all")
test_loader = DataLoader(test_dataset, 50, shuffle=True)

In [10]:
l = []
for b in test_loader:
    x, y = b
    pred = model(x.to(device))
    preds = pred.argmax(dim=1)
    for i, j in enumerate(preds):
        l.append([y.to(device)[i].item(), j.item()])
l.sort(key=lambda i: i[0])

NameError: name 'test_loader' is not defined

In [29]:
import pandas as pd
id = [int(i[0]) for i in l]
cat = [i[1] for i in l]
d = train.class_to_idx
d2 = {str(y):int(x) for x,y in d.items()}
output = pd.DataFrame({"id": id, "category": cat})
output = output.replace({"category": d2})
output

,id,category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
512,512,21
513,513,21
514,514,21
515,515,21


In [30]:
import pandas as pd
import numpy as np
test_label = pd.read_csv("/mnt/storage/home/y263song/test_label.csv", header=0)
test_labels = np.array(test_label["cat"], dtype = "int32")

In [31]:
correct = 0
for i in range(517):
    if output["category"].to_list()[i] == test_labels[i]:
        correct += 1
print(correct/517)

0.7176015473887815


In [23]:
output.to_csv('output.csv', index=False)